<a href="https://colab.research.google.com/github/jaydeepChatrola/Gazal-E-AI/blob/dev/Gazal_e_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!{sys.executable} -m pip install faiss-cpu numpy

In [ ]:
!pip install transformers torch

In [ ]:
from huggingface_hub import login;
login()

In [ ]:
import os
import sys
import json

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype=torch.bfloat16

)


In [ ]:
input_text = "can you write poetry of mirza ghalib"

input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

In [ ]:
chat = [
    { "role": "user", "content": "can you write poetry of mirza ghalib which is hazāroñ ḳhvāhisheñ aisī ki har ḳhvāhish pe dam nikle..." },
]
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
outputs = model.generate(input_ids=inputs.to(model.device), max_new_tokens=150)

print(tokenizer.decode(outputs[0]))

In [115]:
dataset_path = '/content/drive/MyDrive/dataset'

all_file_paths = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        full_file_path = os.path.join(root, file)
        all_file_paths.append(full_file_path)

print(f"Collected {len(all_file_paths)} file paths.")

Collected 1345 file paths.


In [ ]:
documents = []
batch_size = 100

In [ ]:
for i in range(0, len(all_file_paths), batch_size):
    batch = all_file_paths[i:i+batch_size]

    for file_path in batch:
        if os.path.basename(file_path) == ".DS_Store":
            continue

        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
        except Exception as e:
            print(f"Could not read file {file_path}: {e}")
            continue
        print(f"Processing {file_path}")
        documents.append({
            'shayar': file_path.split('/')[5].replace("-", " "),
            'couplet': content
        })

print(f"Collected {len(documents)} documents.")


# Task
To convert the `documents` list to a JSON formatted string and display it, run the following code:

```python
import json

# Convert the documents list to a JSON formatted string
documents_json = json.dumps(documents, indent=4, ensure_ascii=False)

# Display the JSON data
print(documents_json)
```

In [ ]:
documents_json = json.dumps(documents, indent=4, ensure_ascii=False)
print(documents_json)

In [ ]:
import json

# Assuming documents_json contains the full JSON string
parsed_json = json.loads(documents_json)

# Print the first element of the parsed JSON list
if parsed_json:
    print(json.dumps(parsed_json[0], indent=4, ensure_ascii=False))
else:
    print("The JSON is empty.")

In [ ]:
def lines_to_couplets(lines):
    if len(lines) % 2 != 0:
        print("⚠️ Warning: odd number of lines, last line will be ignored")

    couplets = []
    for i in range(0, len(lines) - 1, 2):
        couplets.append(lines[i] + "\n" + lines[i + 1])
    return couplets

def normalize_lines(text: str):
    """
    - removes leading/trailing whitespace
    - splits by newline
    - removes empty lines
    """
    return [
        line.strip()
        for line in text.strip().split("\n")
        if line.strip()
    ]

def build_couplet_records(couplets, shayar, start_index=1):
    records = []
    for idx, text in enumerate(couplets, start=start_index):
        records.append({
            "id": f"{shayar.replace(' ', '_')}_{idx:04d}",
            "shayar": shayar,
            "couplet_index": idx,
            "text": text
        })
    return records


def process_poem_object(poem_obj, start_index=1):
    shayar = poem_obj["shayar"]
    raw_text = poem_obj["couplet"]

    lines = normalize_lines(raw_text)
    couplets = lines_to_couplets(lines)

    return build_couplet_records(couplets, shayar, start_index)




In [ ]:
if isinstance(parsed_json, list):
    final_dataset = []
    counter = 1

    for poem in parsed_json:
        records = process_poem_object(poem, start_index=counter)
        final_dataset.extend(records)
        counter += len(records)

    print(len(final_dataset))



⚠️ Warning: odd number of lines, last line will be ignored
10501


In [ ]:
import json

with open("urdu_sher.json", "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=2)
